In [12]:
#import necessary packages
import pandas as pd
import numpy as np
import os
import stat
from pathlib import Path
import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
import subprocess


In [ ]:
#Area reference data
area_data = pd.read_excel(r'C:\Users\JeevaNithi\Downloads\Areas_in_blore.xlsx')

In [ ]:
#File check module
file_location = r'C:\Users\JeevaNithi\Downloads\zomato'
print(file_location)
files_for_DQ = []
for filename in os.listdir(file_location):
    #print(filename ,os.stat(file_location).st_size,os.stat(filename).st_size)
    if os.stat(file_location).st_size == 0:# only consider if the file contains any data
        print(filename + ": invalid to check DQ")
    else:
        file_ext = filename.rsplit('_',1)[-1].rsplit('.',1)[-1]
        print(file_ext)
        if file_ext != 'csv': # skip the files that are not in .csv format
            print(file_ext+": invalid to check DQ")
        else:
            # process the .csv files
            date = (filename.rsplit('_',1)[-1].rsplit('.',1)[0])
            d = datetime.strptime(date, "%Y%m%d%H%M%S").date()
            datecheck = datetime.today() + relativedelta(months=-6) 
            datecheck = datecheck.strftime('%Y-%m-%d')
            if str(d) >= datecheck:#extract date from file name and take file if it in the prescribed window(taken a time frame of 6 months for POC purpose)
                print(filename)
                files_for_DQ.append(file_location+"\\"+filename)
            else:
                print(filename+" invalid to check DQ")
                print(files_for_DQ) #contains list of file names that are eligible for processing
                #DQ check module
            for file in files_for_DQ:
                print(file)
                df = pd.read_csv(file)
                x=pd.DataFrame()
                x[['contact number 1', 'contact number 2']] = df['phone'].str.split('\r\n', 1, expand=True)#split the phone column into 2 contacts
                phone = x.loc[:, x.columns.str.contains('contact')].apply(lambda x: x.str.strip()).apply(lambda x: x.str.replace('+','',regex=True)) #strip the number and remove + symbol
                df=pd.concat([df, phone], axis=1)
                df['address'] = df['address'].str.replace(r'[^a-zA-Z0-9.,/]+',' ',regex=True) #remove special characters in address
                df['reviews_list'] = df['reviews_list'].str.replace(r'\s+|\\n', ' ', regex=True) #remove special characts in reviews
                df['reviews_list'] = df['reviews_list'].str.replace(r'[^a-zA-Z0-9.,/\'\"]+',' ',regex=True)
                #Invalid order check
                invalid_df = df[(df['address'].isna()) | (df['location'].isna()) & (df['phone'].isna())] #consider if address or location and phone number missing records as invalid orders
                if os.path.isdir(file_location +"\output"):
                    output_path = file_location +"\output"+ "\\" #if output folder is present will be stored there
                else:
                    output_path = file_location + "\\" #if output folder not present will be stored in the same location as the input file
                
                #output_path = file_location + "\\"
                print(output_path + filename +".bad") #store the invalid records as .bad
                invalid_df.to_csv(output_path + filename +".bad")
                
                #Invalid Phone number check
                invalid_phone = df[(df['phone'].isna())]
                invalid_phone.to_csv(output_path + filename +".bad_phone") #for testing purpose stroring invalid phone records as .bad_phone
                #null records
                null = df[df.isnull().any(axis=1)]
                #clean reccords
                clean = df.dropna()
                clean.to_csv(output_path + filename  + ".out") #storing not null records as clean .out files
                
else:
    print("No files Found")